In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import shapiro, kstest, norm
import statsmodels.api as sm
from scipy.stats import gaussian_kde
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from scipy.stats import shapiro, kstest

In [2]:
data = np.loadtxt('Meta2COVID.txt')
df = pd.DataFrame(data, columns=['GD', 'AG', 'MS', 'VA', 'BD', 'HR', 'BP', 'TP', 'Rule'])

df['GUIDELINES'] = 0
df.loc[(df['BD'] >= 2) & (df['TP'] > 37.8),'GUIDELINES'] = 1

# Update HR values
df.loc[df['HR'] <= 55, 'HR'] = 0 #estranho 
df.loc[(df['HR'] > 55) & (df['HR'] <= 77), 'HR'] = 1 #bom a normal
df.loc[df['HR'] > 77, 'HR'] = 2 # mau

# Update BP values
df.loc[df['BP'] <= 114, 'BP'] = 0 # estranho
df.loc[(df['BP'] > 114) & (df['BP'] <= 129), 'BP'] = 1 # normal
df.loc[(df['BP'] > 129) & (df['BP'] <= 139), 'BP'] = 2 # elevado
df.loc[df['BP'] > 139, 'BP'] = 3 # estagio 1 ou 2

# Update TP values
df.loc[df['TP'] <= 35, 'TP'] = 0
df.loc[(df['TP'] > 35) & (df['TP'] <= 37), 'TP'] = 1
df.loc[df['TP'] > 37, 'TP'] = 2

a = df[df['Rule'] == 1]
not_a = df[df['Rule'] == 0]

print(df.dtypes)

GD            float64
AG            float64
MS            float64
VA            float64
BD            float64
HR            float64
BP            float64
TP            float64
Rule          float64
GUIDELINES      int64
dtype: object


In [3]:
# Cálculo de probabilidades a priori
Prob_1 = (df['Rule'] == 1).mean()
Prob_0 = (df['Rule'] == 0).mean()

a['P(Rule=1)'] = Prob_1
not_a['P(Rule=0)'] = Prob_0

# Probabilidades a priori
P_return_home = Prob_0
P_stay_at_hospital = Prob_1


C:\Users\Ze\AppData\Local\Temp\ipykernel_34124\1214145887.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['P(Rule=1)'] = Prob_1
C:\Users\Ze\AppData\Local\Temp\ipykernel_34124\1214145887.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_a['P(Rule=0)'] = Prob_0


In [4]:
print(f'P(Rule=1)={Prob_1}')
print(f'P(Rule=0)={Prob_0}')

P(Rule=1)=0.32166666666666666
P(Rule=0)=0.6783333333333333


In [5]:
Prob_gender_1 = a['GD'].value_counts()/len(a)
Prob_gender_0 = not_a['GD'].value_counts()/len(not_a)

# Prob_ms_1 = a['MS'].value_counts()/len(a)
# Prob_ms_0 = not_a['MS'].value_counts()/len(not_a)

Prob_va_1 = a['VA'].value_counts()/len(a)
Prob_va_0 = not_a['VA'].value_counts()/len(not_a)

Prob_bd_1 = a['BD'].value_counts()/len(a)
Prob_bd_0 = not_a['BD'].value_counts()/len(not_a)

Prob_guid_1 = a['GUIDELINES'].value_counts()/len(a)
Prob_guid_0 = not_a['GUIDELINES'].value_counts()/len(not_a)

# para variables continuas

Prob_hr_1 = a['HR'].value_counts()/len(a)
Prob_hr_0 = not_a['HR'].value_counts()/len(not_a)

Prob_bp_1 = a['BP'].value_counts()/len(a)
Prob_bp_0 = not_a['BP'].value_counts()/len(not_a)

Prob_tp_1 = a['TP'].value_counts()/len(a)
Prob_tp_0 = not_a['TP'].value_counts()/len(not_a)

print(Prob_bd_1)
print(Prob_hr_1)
print(Prob_bp_1)
print(Prob_tp_1)

BD
3.0    0.367876
2.0    0.279793
1.0    0.202073
0.0    0.150259
Name: count, dtype: float64
HR
2.0    0.383420
1.0    0.357513
0.0    0.259067
Name: count, dtype: float64
BP
3.0    0.466321
1.0    0.310881
2.0    0.222798
Name: count, dtype: float64
TP
2.0    0.875648
1.0    0.124352
Name: count, dtype: float64


## Transformar variáveis continuas nas discretas

In [6]:
Prob_0_gender = Prob_gender_0*Prob_0
Prob_1_gender = Prob_gender_1*Prob_1

# Prob_0_ms = Prob_ms_0*Prob_0
# Prob_1_ms = Prob_ms_1*Prob_1

Prob_0_va = Prob_va_0*Prob_0
Prob_1_va = Prob_va_1*Prob_1

Prob_0_bd = Prob_bd_0*Prob_0
Prob_1_bd = Prob_bd_1*Prob_1

Prob_0_guid = Prob_guid_0*Prob_0
Prob_1_guid = Prob_guid_1*Prob_1

# para variables continuas

Prob_0_hr = Prob_hr_0*Prob_0
Prob_1_hr = Prob_hr_1*Prob_1

Prob_0_bp = Prob_bp_0*Prob_0
Prob_1_bp = Prob_bp_1*Prob_1

Prob_0_tp = Prob_tp_0*Prob_0
Prob_1_tp = Prob_tp_1*Prob_1

In [7]:
Prob1_discrete = (Prob_gender_1[df['GD']].values *
                #   Prob_ms_1[df['MS']].values *
                  Prob_va_1[df['VA']].values *
                  Prob_bd_1[df['BD']].values *
                  Prob_guid_1[df['GUIDELINES']].values *
                  Prob_hr_1[df['HR']].values *
                  Prob_bp_1[df['BP']].values *
                  Prob_tp_1[df['TP']].values)

# Update the dataframe with the likelihood for Rule=1
df['Prob_Discrete (Rule=1)'] = Prob1_discrete * Prob_1

# Calculate the likelihood for Rule=0 similarly

# Ensure to check the indexing for continuous variable probability lookup
Prob0_discrete = (Prob_gender_0[df['GD']].values *
                #   Prob_ms_0[df['MS']].values *
                  Prob_va_0[df['VA']].values *
                  Prob_bd_0[df['BD']].values *
                  Prob_guid_0[df['GUIDELINES']].values *
                  Prob_hr_0[df['HR']].values *
                  Prob_bp_0[df['BP']].values *
                  Prob_tp_0[df['TP']].values)

df['Prob_Discrete (Rule=0)'] = Prob0_discrete * Prob_0

df


,GD,AG,MS,VA,BD,HR,BP,TP,Rule,GUIDELINES,Prob_Discrete (Rule=1),Prob_Discrete (Rule=0)
0,0.0,49.148903,1.0,1.0,1.0,2.0,2.0,2.0,0.0,0,0.000178,0.001696
1,0.0,84.766807,1.0,0.0,3.0,2.0,2.0,1.0,1.0,0,0.000182,0.000547
2,1.0,75.031336,0.0,0.0,3.0,2.0,2.0,2.0,1.0,0,0.003295,0.000552
3,1.0,55.087958,0.0,0.0,1.0,0.0,3.0,1.0,0.0,0,0.000363,0.001985
4,0.0,62.550938,1.0,0.0,2.0,2.0,1.0,2.0,1.0,0,0.001358,0.000985
...,...,...,...,...,...,...,...,...,...,...,...,...
595,1.0,68.820531,1.0,0.0,1.0,1.0,2.0,1.0,0.0,0,0.000240,0.001010
596,0.0,44.434241,1.0,0.0,1.0,1.0,3.0,2.0,1.0,0,0.001372,0.004945
597,0.0,79.903478,0.0,1.0,3.0,0.0,1.0,2.0,0.0,1,0.000166,0.000239
598,1.0,63.209868,0.0,0.0,2.0,1.0,2.0,2.0,1.0,1,0.001271,0.000144


In [8]:
def likelihood_z1(row):
    # Verossimilhança ajustada para GD (Gender)
    gender_likelihood_return_home = Prob_gender_0.get(row['GD'], 0)
    gender_likelihood_stay_at_hospital = Prob_gender_1.get(row['GD'], 0)
    
    # Verossimilhança ajustada para MS (Marital Status)
    # marital_status_likelihood_return_home = Prob_ms_0.get(row['MS'], 0)
    # marital_status_likelihood_stay_at_hospital = Prob_ms_1.get(row['MS'], 0)
    
    # Verossimilhança ajustada para VA (Vaccinated)
    vaccinated_likelihood_return_home = Prob_va_0.get(row['VA'], 0)
    vaccinated_likelihood_stay_at_hospital = Prob_va_1.get(row['VA'], 0)
    
    # Verossimilhança ajustada para BD (Breathing Difficulty)
    bd_likelihood_return_home = Prob_bd_0.get(row['BD'], 0)
    bd_likelihood_stay_at_hospital = Prob_bd_1.get(row['BD'], 0)
    
    # Verossimilhança ajustada para GUIDELINES
    guidelines_likelihood_return_home = Prob_guid_0.get(row['GUIDELINES'], 0)
    guidelines_likelihood_stay_at_hospital = Prob_guid_1.get(row['GUIDELINES'], 0)
    
    # Verossimilhança ajustada para hr (Heart Rate)
    hr_likelihood_return_home = Prob_bd_0.get(row['HR'], 0)
    hr_likelihood_stay_at_hospital = Prob_bd_1.get(row['HR'], 0)
    
    # Verossimilhança ajustada para bp (Blood Pressure)
    bp_likelihood_return_home = Prob_bd_0.get(row['BP'], 0)
    bp_likelihood_stay_at_hospital = Prob_bd_1.get(row['BP'], 0)
    
    # Verossimilhança ajustada para tp (Temperature)
    tp_likelihood_return_home = Prob_bd_0.get(row['TP'], 0)
    tp_likelihood_stay_at_hospital = Prob_bd_1.get(row['TP'], 0)
    
    # Combinar todas as verossimilhanças
    likelihood_return_home = (gender_likelihood_return_home * 
                            #   marital_status_likelihood_return_home * 
                              vaccinated_likelihood_return_home * 
                              bd_likelihood_return_home *
                              guidelines_likelihood_return_home*
                              hr_likelihood_return_home*
                              bp_likelihood_return_home*
                              tp_likelihood_return_home)
                              
    likelihood_stay_at_hospital = (gender_likelihood_stay_at_hospital * 
                                #    marital_status_likelihood_stay_at_hospital * 
                                   vaccinated_likelihood_stay_at_hospital * 
                                   bd_likelihood_stay_at_hospital *
                                   guidelines_likelihood_stay_at_hospital*
                                   hr_likelihood_stay_at_hospital*
                                   bp_likelihood_stay_at_hospital*
                                   tp_likelihood_stay_at_hospital)
    
    return likelihood_return_home, likelihood_stay_at_hospital


In [9]:
# Estatísticas descritivas para Age (AG)
mean_age_return_home = not_a['AG'].mean()
std_age_return_home = not_a['AG'].std()
mean_age_stay_at_hospital = a['AG'].mean()
std_age_stay_at_hospital = a['AG'].std()

print(f"Mean Age for Return Home: {mean_age_return_home}, Std: {std_age_return_home}")
print(f"Mean Age for Stay at Hospital: {mean_age_stay_at_hospital}, Std: {std_age_stay_at_hospital}")

Mean Age for Return Home: 65.7064469066339, Std: 12.376304195594047
Mean Age for Stay at Hospital: 64.66585355440415, Std: 12.271284945138401


In [10]:
# Função para calcular a verossimilhança baseada na distribuição Gaussiana
def likelihood_age(age):
    likelihood_return_home = norm.pdf(age, mean_age_return_home, std_age_return_home)
    likelihood_stay_at_hospital = norm.pdf(age, mean_age_stay_at_hospital, std_age_stay_at_hospital)
    return likelihood_return_home, likelihood_stay_at_hospital

In [11]:
# Função para calcular a probabilidade a posteriori combinada
def combined_posterior_probabilities(row):
    # Calcular verossimilhança de Z1
    z1_likelihood_return_home, z1_likelihood_stay_at_hospital = likelihood_z1(row)
    
    # Calcular probabilidade a posteriori de Z2
    z2_posterior_return_home, z2_posterior_stay_at_hospital = posterior_probabilities(row)
    
    # Calcular numeradores das probabilidades a posteriori combinadas
    combined_posterior_return_home = z1_likelihood_return_home * z2_posterior_return_home
    combined_posterior_stay_at_hospital = z1_likelihood_stay_at_hospital * z2_posterior_stay_at_hospital
    
    # Calcular o denominador (evidência)
    evidence = combined_posterior_return_home + combined_posterior_stay_at_hospital
    
    # Calcular as probabilidades a posteriori combinadas
    posterior_return_home = combined_posterior_return_home / evidence
    posterior_stay_at_hospital = combined_posterior_stay_at_hospital / evidence
    
    return posterior_return_home, posterior_stay_at_hospital


In [12]:
# Função geral de verossimilhança para z2 (incluindo Age e variáveis contínuas usando KDE)
def likelihood_z2(row):
    # Verossimilhança para Age (Gaussiana)
    age_likelihood_return_home, age_likelihood_stay_at_hospital = likelihood_age(row['AG'])

    # Combinar todas as verossimilhanças (multiplicação das verossimilhanças)
    likelihood_return_home = age_likelihood_return_home
    likelihood_stay_at_hospital = age_likelihood_stay_at_hospital
    
    return likelihood_return_home, likelihood_stay_at_hospital

In [13]:
# Função para calcular a probabilidade a posteriori total
def posterior_probabilities_total(row):
    z1_likelihood_return_home, z1_likelihood_stay_at_hospital = likelihood_z1(row)
    z2_likelihood_return_home, z2_likelihood_stay_at_hospital = likelihood_z2(row)
    
    posterior_return_home = z1_likelihood_return_home * z2_likelihood_return_home * P_return_home
    posterior_stay_at_hospital = z1_likelihood_stay_at_hospital * z2_likelihood_stay_at_hospital * P_stay_at_hospital
    
    evidence = posterior_return_home + posterior_stay_at_hospital
    
    posterior_return_home /= evidence
    posterior_stay_at_hospital /= evidence
    
    return posterior_return_home, posterior_stay_at_hospital

# Aplicar a função de probabilidade a posteriori total para uma linha do DataFrame
row_example = df.iloc[0]  # Exemplo de uma linha do DataFrame
posterior_return_home, posterior_stay_at_hospital = posterior_probabilities_total(row_example)
print(f"Posterior probability of returning home for the given row: {posterior_return_home}")
print(f"Posterior probability of staying at hospital for the given row: {posterior_stay_at_hospital}")


Posterior probability of returning home for the given row: 0.9194499528855907
Posterior probability of staying at hospital for the given row: 0.08055004711440929


In [14]:
# Função para tomar a decisão final baseada nas probabilidades a posteriori totais
def make_final_decision(row):
    posterior_return_home, posterior_stay_at_hospital = posterior_probabilities_total(row)
    return 0 if posterior_return_home > posterior_stay_at_hospital else 1

# Aplicar a função de decisão ao DataFrame
df['Final Decision'] = df.apply(make_final_decision, axis=1)

# Mostrar as primeiras linhas do DataFrame com a decisão final
print(df.head())


    GD         AG   MS   VA   BD   HR   BP   TP  Rule  GUIDELINES  \
0  0.0  49.148903  1.0  1.0  1.0  2.0  2.0  2.0   0.0           0   
1  0.0  84.766807  1.0  0.0  3.0  2.0  2.0  1.0   1.0           0   
2  1.0  75.031336  0.0  0.0  3.0  2.0  2.0  2.0   1.0           0   
3  1.0  55.087958  0.0  0.0  1.0  0.0  3.0  1.0   0.0           0   
4  0.0  62.550938  1.0  0.0  2.0  2.0  1.0  2.0   1.0           0   

   Prob_Discrete (Rule=1)  Prob_Discrete (Rule=0)  Final Decision  
0                0.000178                0.001696               0  
1                0.000182                0.000547               0  
2                0.003295                0.000552               1  
3                0.000363                0.001985               0  
4                0.001358                0.000985               0  


In [15]:
# Definir variáveis para armazenar as métricas de desempenho
accuracies = []
precisions = []
recalls = []
f1_scores = []

# Configurar a validação cruzada
kf = KFold(n_splits=5, shuffle=True, random_state=1)

# Realizar a validação cruzada
for train_index, test_index in kf.split(df):
    train_df, test_df = df.iloc[train_index], df.iloc[test_index]
    
    # Aplicar a função de decisão no conjunto de treino
    train_df['Predicted'] = train_df.apply(make_final_decision, axis=1)
    
    # Calcular as métricas de desempenho no conjunto de teste
    y_true = test_df['Rule']
    y_pred = test_df.apply(make_final_decision, axis=1)
    
    accuracies.append(accuracy_score(y_true, y_pred))
    precisions.append(precision_score(y_true, y_pred))
    recalls.append(recall_score(y_true, y_pred))
    f1_scores.append(f1_score(y_true, y_pred))

# Calcular e imprimir as médias das métricas de desempenho
print(f"Acurácia média: {np.mean(accuracies):.4f}")
print(f"Precisão média: {np.mean(precisions):.4f}")
print(f"Recall médio: {np.mean(recalls):.4f}")
print(f"F1-score médio: {np.mean(f1_scores):.4f}")


C:\Users\Ze\AppData\Local\Temp\ipykernel_34124\172852607.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['Predicted'] = train_df.apply(make_final_decision, axis=1)
C:\Users\Ze\AppData\Local\Temp\ipykernel_34124\172852607.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['Predicted'] = train_df.apply(make_final_decision, axis=1)
C:\Users\Ze\AppData\Local\Temp\ipykernel_34124\172852607.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


Acurácia média: 0.8017
Precisão média: 0.7388
Recall médio: 0.5796
F1-score médio: 0.6483


C:\Users\Ze\AppData\Local\Temp\ipykernel_34124\172852607.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['Predicted'] = train_df.apply(make_final_decision, axis=1)
C:\Users\Ze\AppData\Local\Temp\ipykernel_34124\172852607.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['Predicted'] = train_df.apply(make_final_decision, axis=1)
